In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import os
from pathlib import Path

: 

## Data Prep

In [ ]:
# Define the base directory where your data is located
current_dir = Path.cwd()
print(current_dir)
data = (Path(current_dir) / "data").resolve()
# Paths for the shapefiles
divisions_path = os.path.join(data, "USA_Divisions", "usa_divisions.shp")
counties_path = os.path.join(data, "USA_Counties", "usa_counties.shp")
states_path = os.path.join(data, "USA_States", "usa_states.shp")

# Reading the shapefiles using geopandas
divisions = gpd.read_file(divisions_path)
counties = gpd.read_file(counties_path)
states = gpd.read_file(states_path)

: 

In [ ]:
print(divisions)

: 

In [ ]:
print(states.columns)

: 

In [ ]:
print(counties["CROP_ACR17"].describe())

: 

Creating Counties DataFrame

In [ ]:
counties_df = pd.DataFrame()
counties_df["FIPS"] = counties["FIPS"]
counties_df["x1"] = counties["POP_SQMI"]
counties_df["x2"] = counties["OWNER_OCC"] / counties["RENTER_OCC"]
counties_df["x3"] = counties["CROP_ACR17"]
counties_df["geometry"] = counties["geometry"]

: 

Preparing State and Divisions dataframes

In [ ]:
states_df = pd.DataFrame()
states_df["FIPS"] = states["STATEFP"]
states_df["geometry"] = states["geometry"]

divisions_df = pd.DataFrame()
divisions_df["FIPS"] = divisions.index
divisions_df["geometry"] = divisions["geometry"]

: 

In [ ]:
# Assuming 'states' and 'divisions' are already defined and have geometry information

# Convert 'states_df' to a GeoDataFrame
states_df = pd.DataFrame()
states_df["FIPS"] = states["STATEFP"]
states_df["geometry"] = states["geometry"]
states_gdf = gpd.GeoDataFrame(states_df, geometry="geometry")

counties_gdf = gpd.GeoDataFrame(counties_df, geometry="geometry")

# Convert 'divisions_df' to a GeoDataFrame
divisions_df = pd.DataFrame()
divisions_df["FIPS"] = divisions.index
divisions_df["geometry"] = divisions["geometry"]
divisions_gdf = gpd.GeoDataFrame(divisions_df, geometry="geometry")

# Ensure they have the correct coordinate reference system (CRS)
# Example: setting to WGS 84 (EPSG:4326)
states_gdf.crs = "EPSG:4326"
divisions_gdf.crs = "EPSG:4326"

# Now you can perform spatial operations on 'states_gdf' and 'divisions_gdf'

: 

In [ ]:
# Spatial join Counties with States
states_counties = gpd.sjoin(counties_gdf, states_gdf, how="inner", op="within")
print(states_counties.columns)
# Aggregate/Average Data for States
states_agg = (
    states_counties.groupby("FIPS_right")
    .agg(
        x1=("x1", "mean"),  # Average population density
        x2=(
            "x2",
            "mean",
        ),  # Average owner/renter ratio
        x3=("x3", "sum"),  # Sum of crop acreage
    )
    .reset_index()
)

# Spatial join Counties with Divisions
divisions_counties = gpd.sjoin(counties_gdf, divisions_gdf, how="inner", op="within")

# Aggregate/Average Data for Divisions
divisions_agg = (
    divisions_counties.groupby("FIPS_right")
    .agg(
        x1=("x1", "mean"),
        x2=("x2", "mean"),
        x3=("x3", "sum"),
    )
    .reset_index()
)

: 

In [ ]:
states_agg["geometry"] = states["geometry"]
states_agg["names"] = states["NAME"]
states_agg = states_agg.rename(columns={"FIPS_right": "FIPS"})
divisions_agg["geometry"] = divisions["geometry"]
divisions_agg["names"] = divisions["division"]
divisions_agg = divisions_agg.rename(columns={"FIPS_right": "FIPS"})

: 

Renaming

In [ ]:
counties = counties_df
states = states_agg
divisions = divisions_agg

: 

In [ ]:
print(counties)
print(states)
print(divisions)

: 

## Stochastic Model

In [ ]:
def logistic_function_with_noise_normalized(x1, x2, x3, a, b, y, sigma, steps=100):
    """
    Generate stochastic paths using a logistic function with added noise, and normalize
    the results to get probabilities that sum to 1.

    Parameters:
    - x1, x2, x3: Variables for population density, owner/renter ratio, and crop acreage.
    - a, b, y: Parameters weighting the importance of x1, x2, and x3.
    - sigma: Standard deviation of the noise term.
    - steps: Number of steps in the stochastic path.

    Returns:
    - A numpy array containing normalized probabilities for each step.
    """
    path = np.zeros(steps)
    for i in range(steps):
        e = np.random.normal(0, sigma**2)  # Noise term
        logistic_val = 1 / (1 + np.exp(-(a * x1 + b * x2 + y * x3)))
        path[i] = logistic_val + e

    # Normalize the path to ensure the sum of probabilities equals 1
    normalized_path = np.maximum(path, 0)  # Ensure all probabilities are non-negative
    sum_path = np.sum(normalized_path)
    if sum_path > 0:
        normalized_path = normalized_path / sum_path
    else:
        # In case sum_path is 0 or negative due to noise, distribute probabilities evenly
        normalized_path = np.ones(steps) / steps

    return normalized_path

: 

In [ ]:
def generate_stochastic_paths_df(gdf, a, b, y, sigma):
    data_for_new_df = []
    for index, row in gdf.iterrows():
        path = logistic_function_with_noise_normalized(
            row["x1"], row["x2"], row["x3"], a, b, y, sigma
        )
        data_for_new_df.append({"FIPS": row["FIPS"], "stochastic_paths": path.tolist()})
    return pd.DataFrame(data_for_new_df)

: 

In [ ]:
def plot_pdf_for_steps(stochastic_paths_df, name, steps_to_plot=[0, 49, 99]):
    all_paths = np.array(stochastic_paths_df["stochastic_paths"].tolist())
    transposed_paths = (
        all_paths.T
    )  # Transpose so each row represents a step across all paths

    # Step 2: Plot the PDF for each step
    steps_to_plot = [
        0,
        49,
        99,
    ]  # For example, plot the PDF for the first, middle, and last step
    for step in steps_to_plot:
        probabilities = transposed_paths[step]

        # Compute the histogram
        density, bins = np.histogram(probabilities, bins=30, density=True)
        # Compute bin centers
        bin_centers = (bins[:-1] + bins[1:]) / 2

        # Plot the PDF
        plt.plot(bin_centers, density, label=f"Step {step + 1}")

    plt.xlabel("Probability")
    plt.xticks(rotation=45)  # Rotate labels to prevent overlap
    plt.ylabel("Density")
    plt.title(f"Probability Density Function for {name} at Selected Steps")
    plt.legend()
    plt.show()

: 

In [ ]:
# Example parameters
a, b, y, sigma = 0.1, 0.2, 0.3, 0.05

# Generate stochastic paths DataFrame for counties
counties_stochastic_df = generate_stochastic_paths_df(counties, a, b, y, sigma)
# Plot PDF for counties
plot_pdf_for_steps(counties_stochastic_df, name="Counties")

# Repeat similarly for states and divisions
states_stochastic_df = generate_stochastic_paths_df(states, a, b, y, sigma)
plot_pdf_for_steps(states_stochastic_df, name="States")

divisions_stochastic_df = generate_stochastic_paths_df(divisions, a, b, y, sigma)
plot_pdf_for_steps(divisions_stochastic_df, name="Divisions")

: 

In [ ]:
# Define the directory to save the shapefiles
save_dir = Path.cwd() / "stochastic_paths"
save_dir.mkdir(exist_ok=True)  # Create the directory if it doesn't exist


# Function to save GeoDataFrames to the specified directory
def save_gdf(gdf, name, directory=save_dir):
    """
    Save a GeoDataFrame as a shapefile in the specified directory.

    Parameters:
    - gdf: The GeoDataFrame to save.
    - name: The name for the saved file (without extension).
    - directory: The directory where the shapefile will be saved.
    """
    file_path = directory / f"{name}.csv"
    gdf.to_csv(file_path, index=False)


# Save the GeoDataFrames
save_gdf(counties_stochastic_df, "counties_stochastic")
save_gdf(states_stochastic_df, "states_stochastic")
save_gdf(divisions_stochastic_df, "divisions_stochastic")

: 